In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
import requests 
from bs4 import BeautifulSoup
import xml
import folium
from geopy.geocoders import Nominatim

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_V').text
soup=BeautifulSoup(source,'lxml')
table = soup.find('table')
table_contents = []

#Defining dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood']

#create the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

cells = table.find_all('td')
#print(cells)

# append the data into the respective lists
for row in cells:
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.b.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        try: 
            cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        except:
            cell['Neighborhood'] = (row.span.text).split('(')[0]
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
#df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
 #                                            'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
  #                                           'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
   #                                          'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [3]:
sorted = df.sort_values(by=['PostalCode'])

In [4]:
vancouver_final = df[df['Borough'].str.contains('Vancouver',regex=False)]
vancouver_final.head()

,PostalCode,Borough,Neighborhood
5,V6A,Vancouver,"Strathcona, Chinatown, Downtown Eastside"
14,V6B,Vancouver,"NE Downtown, Gastown, Harbour Centre, Internat..."
23,V6C,Vancouver,"Waterfront, Coal Harbour, Canada Place"
32,V6E,Vancouver,"SE West End, Davie Village"
41,V6G,Vancouver,"NW West End, Stanley Park"


In [5]:
data = [['V6A', 'Vancouver', 'Chinatown', 49.2777, -123.0937]]
chi_town = pd.DataFrame(data, columns = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'])

In [6]:
CLIENT_ID = 'IIGO3V11Y20O1XCU2YREWZNTATYPCSWP1LHY43ZGDF0TX1LA' # your Foursquare ID
CLIENT_SECRET = '2OLYD1ZW4G2MTVIT5KR5PCMAFMVON0AJFPGQNONAKTXCBYT3' # your Foursquare Secret
ACCESS_TOKEN = '3CH5BPMI23GM44VZPNOMU10ORGRUNS3FE1QKQNWZIJ0MPTLG' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

#IIGO3V11Y20O1XCU2YREWZNTATYPCSWP1LHY43ZGDF0TX1LA
#https://www.google.com/?code=BP5TTGERMFVBYLQCGAQIVZJQVRRLLZOPKIOCMGH0HL4I0UG0#_=_
#https://foursquare.com/oauth2/access_token?client_id=IIGO3V11Y20O1XCU2YREWZNTATYPCSWP1LHY43ZGDF0TX1LA&client_secret=2OLYD1ZW4G2MTVIT5KR5PCMAFMVON0AJFPGQNONAKTXCBYT3&grant_type=authorization_code&redirect_uri=https://www.google.com/?code=BP5TTGERMFVBYLQCGAQIVZJQVRRLLZOPKIOCMGH0HL4I0UG0#_=_&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork-21253531&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ

Your credentails:
CLIENT_ID: IIGO3V11Y20O1XCU2YREWZNTATYPCSWP1LHY43ZGDF0TX1LA
CLIENT_SECRET:2OLYD1ZW4G2MTVIT5KR5PCMAFMVON0AJFPGQNONAKTXCBYT3


In [7]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 'id', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
chi_town_venues = getNearbyVenues(names=chi_town['Neighborhood'],
                                   latitudes=chi_town['Latitude'],
                                   longitudes=chi_town['Longitude']
                                  )



Chinatown


In [9]:
print('There are {} unique categories.'.format(len(chi_town_venues['Venue Category'].unique())))

There are 22 unique categories.


In [10]:
# one hot encoding
chi_town_onehot = pd.get_dummies(chi_town_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
chi_town_onehot['neighborhood'] = chi_town_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [chi_town_onehot.columns[-1]] + list(chi_town_onehot.columns[:-1])
chi_town_onehot = chi_town_onehot[fixed_columns]

chi_town_onehot.head()

,neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bar,Café,Coffee Shop,Deli / Bodega,Dessert Shop,Dive Bar,Food Truck,Gastropub,Gourmet Shop,Japanese Restaurant,Lounge,Noodle House,Park,Pie Shop,Pizza Place,Poke Place,Sandwich Place,Seafood Restaurant,Tea Room
0,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,Chinatown,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Chinatown,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,Chinatown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [11]:
chi_town_grouped = chi_town_onehot.groupby('neighborhood').mean().reset_index()
chi_town_grouped

,neighborhood,American Restaurant,Asian Restaurant,BBQ Joint,Bar,Café,Coffee Shop,Deli / Bodega,Dessert Shop,Dive Bar,Food Truck,Gastropub,Gourmet Shop,Japanese Restaurant,Lounge,Noodle House,Park,Pie Shop,Pizza Place,Poke Place,Sandwich Place,Seafood Restaurant,Tea Room
0,Chinatown,0.033333,0.1,0.033333,0.033333,0.1,0.066667,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.033333,0.066667,0.033333,0.1,0.033333,0.033333


In [12]:
num_top_venues = 5

for hood in chi_town_grouped['neighborhood']:
    print("----"+hood+"----")
    temp = chi_town_grouped[chi_town_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Chinatown----
              venue  freq
0    Sandwich Place  0.10
1              Café  0.10
2  Asian Restaurant  0.10
3       Coffee Shop  0.07
4       Pizza Place  0.07




In [13]:
#Locate the lat an lon of Toronto
address = 'Vancouver, BC, Canada'

geolocator = Nominatim(user_agent="Vancouver_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Vancouver {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Vancouver 49.2608724, -123.1139529.


In [14]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'lxml')
table = soup.find('table')
table_contents = []


#Defining dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood']

#create the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

cells = table.find_all('td')
#print(cells)

# append the data into the respective lists
for row in cells:
    cell = {}
    if row.span.text == 'Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [15]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [16]:
sorted = df.sort_values(by=['PostalCode'])

In [17]:
latlon = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

In [18]:
latlon.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
latlon.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [19]:
canada_final = pd.merge(sorted, latlon, on='PostalCode')

In [20]:
#Extract just the neighborhoods located in Toronto
toronto_final = canada_final[canada_final['Borough'].str.contains('Toronto',regex=False)]
toronto_final.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
40,M4J,East York/East Toronto,The Danforth East,43.685347,-79.338106
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923


In [21]:
#Locate the lat an lon of Toronto
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


In [22]:
toronto_venues = getNearbyVenues(names=toronto_final['Neighborhood'],
                                   latitudes=toronto_final['Latitude'],
                                   longitudes=toronto_final['Longitude']
                                  )

The Beaches
The Danforth  East
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Enclave of M5E
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction 

In [23]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

In [24]:
toronto_grouped = toronto_onehot.groupby('neighborhood').mean().reset_index()

In [25]:
result = toronto_grouped.append(chi_town_grouped)

In [26]:
final_df = result.fillna(0)

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [28]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['neighborhood'] = final_df['neighborhood']

for ind in np.arange(final_df.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(final_df.iloc[ind, :], num_top_venues)



In [29]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 12

toronto_grouped_clustering = final_df.drop('neighborhood', 1)
toronto_grouped_clustering
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 8, 3, 8, 9, 3, 8, 3, 3, 7], dtype=int32)

In [30]:
# add clustering labels
final_df.insert(0, 'Cluster Labels', kmeans.labels_)


In [31]:
count = 0
for column in final_df['Cluster Labels']:
    if column == final_df.iloc[39,0]:
        count+=1
print(count)

10


In [32]:
candidates = final_df[final_df['Cluster Labels'] == final_df.iloc[39,0]]

In [33]:
candidates

,Cluster Labels,neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health Food Store,Historic Site,History Museum,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Spa,Speakeasy,Sporting Goods Shop,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Dive Bar,Pie Shop
0,0,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.033333,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.033333,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.033333,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.033333,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.033333,0.066667,0.033333,0.000000,0.000000,0.000000,0.0,0.033333,0.000000,0.033333,0.0,0.000000,0.000000,0.033333,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.066667,0.000000,0.0,0.033333,0.00,0.0,0.000000,0.033333,0.033333,0.00,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.033333,0.033333,0.000000,0.0,0.0,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.033333,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.033333,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.033333,0.0,0.0,0.0,0.000000,0.0,0.066667,0.000000,0.0,0.0,0.0,0.0,0.0,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000

In [34]:
candidates.shape

(10, 196)

In [35]:
#removing empty columns
for column in candidates:
    try:
        if sum(candidates[column]) == 0:
            del candidates[column]
    except:
        continue

candidates.shape

(10, 111)

In [45]:
toronto_csv1 = pd.read_csv('toronto_final.csv')

In [46]:
toronto_final1 = toronto_csv1
for index,row in toronto_final1.iterrows():
    if row['Neighborhood'] not in candidates.values:
        toronto_final1 = toronto_final1.drop(index=index)

In [47]:
toronto_final1

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
4,43,M4M,East Toronto,Studio District,43.659526,-79.340923
12,51,M4X,Downtown Toronto,"St. James Town, Cabbagetown",43.667967,-79.367675
15,54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
16,55,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
17,56,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
26,66,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049
27,67,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049
29,69,M5W,Downtown Toronto Stn A,Enclave of M5E,43.646435,-79.374846
35,82,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763


In [48]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map by neighborhoods
for lat, lng, borough, neighborhood in zip(toronto_final1['Latitude'], toronto_final1['Longitude'], toronto_final1['Borough'], toronto_final1['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [36]:
candidates

,neighborhood,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brewery,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Deli / Bodega,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Latin American Restaurant,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Park,Pet Store,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Dive Bar,Pie Shop
0,Berczy Park,0.000000,0.00,0.033333,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.00,0.000000,0.000000,0.033333,0.066667,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.033333,0.00,0.000000,0.033333,0.033333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11,Enclave of M5E,0.000000,0.00,0.033333,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.033333,0.00,0.033333,0.033333,0.033333,0.00,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14,"Garden District, Ryerson",0.000000,0.00,0.033333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.100000,0.00,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.00,0.000000,0.033333,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.

In [37]:
#

# set number of clusters
kclusters = 4

second_clusters = candidates.drop('neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(second_clusters)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 3, 0, 1, 1, 3, 2, 2, 0, 0], dtype=int32)

In [38]:
candidates.insert(0, 'Cluster Labels', kmeans.labels_)

In [39]:
candidates

,Cluster Labels,neighborhood,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brewery,Burger Joint,Burrito Place,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Deli / Bodega,Dessert Shop,Diner,Discount Store,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Latin American Restaurant,Liquor Store,Lounge,Market,Mexican Restaurant,Middle Eastern Restaurant,Museum,Music Venue,Neighborhood,New American Restaurant,Noodle House,Organic Grocery,Park,Pet Store,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Speakeasy,Sporting Goods Shop,Stationery Store,Steakhouse,Sushi Restaurant,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio,Dive Bar,Pie Shop
0,3,Berczy Park,0.000000,0.00,0.033333,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.033333,0.066667,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.00,0.000000,0.000000,0.033333,0.066667,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.033333,0.00,0.000000,0.033333,0.033333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
11,3,Enclave of M5E,0.000000,0.00,0.033333,0.000000,0.0,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.066667,0.000000,0.033333,0.00,0.033333,0.033333,0.033333,0.00,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.066667,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
14,0,"Garden District, Ryerson",0.000000,0.00,0.033333,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.000000,0.033333,0.033333,0.000000,0.100000,0.00,0.000000,0.000000,0.066667,0.000000,0.033333,0.000000,0.000000,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.00,0.000000,0.033333,0.000000,0.033333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000

In [40]:
#Locate the lat an lon of Toronto
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto 43.6534817, -79.3839347.


In [41]:
toronto_csv = pd.read_csv('toronto_final.csv')

In [42]:
toronto_final = toronto_csv
for index,row in toronto_final.iterrows():
    if row['Neighborhood'] != 'University of Toronto, Harbord' and row['Neighborhood'] != 'Garden District, Ryerson':
        toronto_final = toronto_final.drop(index=index)

In [43]:
toronto_final

,Unnamed: 0,PostalCode,Borough,Neighborhood,Latitude,Longitude
15,54,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
26,66,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049


In [44]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map by neighborhoods
for lat, lng, borough, neighborhood in zip(toronto_final['Latitude'], toronto_final['Longitude'], toronto_final['Borough'], toronto_final['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto